**TODO**
- DB Anbindung
- Bugfix while Schleife

Wir werden anhand des Iris Datensatzes eine einfache Predictive Pipeline in Python Demonstrieren.

* Irirs Datensatz wird aufgespallten Streaming Data, Training Data
* Auf dem trainings Teil wird ein Modell Trainiert
* Der Streaming Teil wird über einen Producer in Kafka gestreamt unter dem Topic 'iris'
* Der Consumer lauscht auf das Topic 'iris' und nimmt die Messages entgegen, betreibt inference und schreibt die Daten und das Modell Ergebniss in die MongoDB

In [ ]:
from sklearn.datasets        import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors       import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from confluent_kafka         import Producer, Consumer
import pandas as pd
import numpy  as np
import threading
import logging

In [ ]:
VERBOSE = False

**Laden und vorbereiten der Daten**

In [ ]:
data = load_iris()
if VERBOSE:
    print(data.DESCR)

In [ ]:
feature_names = [e.replace(' ','_').replace('(cm)','in_cm') for e in data.feature_names]
X_stream, X_train, y_stream, y_train = train_test_split(data['data'], data['target'])
X_stream = pd.DataFrame(X_stream, columns=feature_names)
X_train  = pd.DataFrame(X_train, columns=feature_names)

**Trainieren des Modells auf historischen Daten**

In [ ]:
if VERBOSE:
    for n_neighbors in range(1,X_train.shape[0]-5):
        knn = KNeighborsClassifier(n_neighbors=n_neighbors)
        print("%s : %s" % (n_neighbors,
                           cross_val_score(knn, X_train, y_train, cv=10, scoring='accuracy').mean()))

In [ ]:
n_neighbors = 17
knn = KNeighborsClassifier(n_neighbors=n_neighbors)
knn.fit(X_train,y_train)

**Erstellen eines Kafka Producers**

In [ ]:
# Global Config
TOPIC = 'iris'
BOOTSTRAP_SERVERS = 'kafka1:19092,kafka2:29092,kafka3:39092'

In [ ]:
def delivery_callback(err, msg):
    if err:
        print('%% Message failed delivery: %s\n' % err)
    else:
        if VERBOSE:
            print('%% Message delivered to %s [%d] @ %o\n' %
                             (msg.topic(), msg.partition(), msg.offset()))

In [ ]:
p = Producer({'bootstrap.servers': BOOTSTRAP_SERVERS})

In [ ]:
def push_msgs():
    for msg in msgs:
        p.produce(TOPIC, msg, callback=delivery_callback)
        p.poll(10)
    p.flush()

*Vorbereiten der Nachrichten*

In [ ]:
msgs = [ str(e) for e in eval(X_stream.to_json(orient='records'))]

** Erstellen eines Kafka Consumers ** 

In [ ]:
config_dict = {
        'bootstrap.servers': BOOTSTRAP_SERVERS,
        'group.id': 'mygroup', 
        'default.topic.config': {
            'auto.offset.reset': 'smallest'
        }
}

c = Consumer(config_dict)
c.subscribe([TOPIC])

In [ ]:
def pull_and_process_msg():
    msg = c.poll()
    if not msg.value():
        raw = eval(msg.value())
        model_input = np.array([raw[k] for k in list(X_train.columns)]).reshape(1, -1)
        y_ = knn.predict(model_input)[0]
        if VERBOSE:
            print("Input %s => Prediction %s" % (raw, y_))
        #write to db
        return True
    logging.error("Can't eval %s" % msg.value())
    return False

In [ ]:
producer_thread = threading.Thread(target=push_msgs)
producer_thread.start()

end_of_data = False

while not end_of_data:
    if not producer_thread.is_alive and not pull_and_process_msg():
        if VERBOSE:
            print('Producer running: ', producer_thread.is_alive())
        end_of_data = True